<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

# shape of data is imporotant!!

In [1]:
# TODO - Words, words, mere words, no matter from the heart.

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from keras.utils.data_utils import get_file
import numpy as np
import pandas as pd
import requests
import random
import sys
import io
import os

Using TensorFlow backend.


In [2]:
path = get_file('100-0.txt',origin='https://www.gutenberg.org/files/100/100-0.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

5783552/5777367 [==============================] - 4s 1us/step
corpus length: 5573152


In [21]:
# clearning the text a bit

text = text.replace("\n", ' ')
text = text.replace("\ufeff", '')
text[:4000]

' project gutenberg’s the complete works of william shakespeare, by william shakespeare  this ebook is for the use of anyone anywhere in the united states and most other parts of the world at no cost and with almost no restrictions whatsoever.  you may copy it, give it away or re-use it under the terms of the project gutenberg license included with this ebook or online at www.gutenberg.org.  if you are not located in the united states, you’ll have to check the laws of the country where you are located before using this ebook.   title: the complete works of william shakespeare  author: william shakespeare  release date: january 1994 [ebook #100] last updated: november 7, 2019  language: english  character set encoding: utf-8  *** start of this project gutenberg ebook the complete works of william shakespeare ***     the complete works of william shakespeare    by william shakespeare           contents                   the sonnets                 all’s well that ends well               

In [48]:
# my aim is to train isolate the works so that the model learns specifically from 
# shakespear, and not legal, nor gutenberg related text

# finding the begining of the actual works
text.index('shakespear ***')

text = text[4000:]

In [52]:
# finding the end of the shakespears actual works

text.index('* conntent')

In [50]:
# removing tail end

text = text[:5549291]
text[-1000:]

'he drops the stalk, and in the breach appears   green-dropping sap, which she compares to tears.  “poor flower,” quoth she, “this was thy father’s guise, sweet issue of a more sweet-smelling sire, for every little grief to wet his eyes, to grow unto himself was his desire,                1180   and so ’tis thine; but know, it is as good   to wither in my breast as in his blood.  “here was thy father’s bed, here in my breast; thou art the next of blood, and ’tis thy right:     1184 lo in this hollow cradle take thy rest, my throbbing heart shall rock thee day and night:   there shall not be one minute in an hour   wherein i will not kiss my sweet love’s flower.”  thus weary of the world, away she hies,             1189 and yokes her silver doves; by whose swift aid their mistress mounted through the empty skies, in her light chariot quickly is convey’d;           1192   holding their course to paphos, where their queen   means to immure herself and not be seen.       finis             

In [49]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 72


In [57]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 5

encoded = [char_indices[c] for c in text]

sequences = []
next_char = []

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])

print('sequences: ', len(sequences))

sequences:  1109851


In [59]:
sequences[0]

[49,
 37,
 34,
 1,
 48,
 44,
 43,
 43,
 34,
 49,
 48,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 13,
 1,
 1,
 35,
 47,
 44,
 42]

In [62]:
# creatgin x & y

x = np.zeros((len(sequences), maxlen, len(chars)))
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i, t, char] = 1
    y[i, next_char[i]] = 1

In [63]:
x.shape

(1109851, 40, 72)

In [64]:
y.shape

(1109851, 72)

In [65]:
# build model

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [66]:
def sample(preds):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_pred / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [68]:
def on_epoch_end(epoch, _):
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1
        pred = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = indices_chars[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()
    
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:
model.fit(x, y,
          batch_size=32,
          epochs=10,
          callbacks=[print_callback])

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN